In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel, SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [3]:
my_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
my_data

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.000000,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.000000,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,1.0,4.000000,0.0,0.0,39.0,983.138889,0.015385,0.017599,0.0,0.0,...,2.0,6.0,3.0,Returning_Visitor,Female,Deny,NaN,Married,0.0,True
14727,8.0,117.023809,2.0,57.0,11.0,252.892857,0.000000,0.011078,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Not Specified,ALL,Diploma,Married,0.0,True
14728,2.0,75.600000,2.0,652.8,10.0,1143.666667,0.000000,0.023333,0.0,0.0,...,2.0,4.0,2.0,Returning_Visitor,Not Specified,Required,Others,Single,0.0,True
14729,0.0,0.000000,0.0,0.0,6.0,1057.000000,0.000000,0.033333,0.0,0.0,...,4.0,4.0,1.0,Returning_Visitor,Not Specified,Required,Not Specified,Married,0.0,True


In [4]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   GoogleMetric:Bounce Rates        14580 non-null  float64
 7   GoogleMetric:Exit Rates          14602 non-null  float64
 8   GoogleMetric:Page Values         14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

In [5]:
my_data.shape

(14731, 22)

In [6]:
my_data.columns

Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase',
       'Month_SeasonalPurchase', 'OS', 'SearchEngine', 'Zone',
       'Type of Traffic', 'CustomerType', 'Gender', 'Cookies Setting',
       'Education', 'Marital Status', 'WeekendPurchase', 'Made_Purchase'],
      dtype='object')

In [7]:
my_data.describe()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
count,14578.000000,14581.000000,14578.000000,14596.000000,14608.000000,14564.000000,14580.000000,14602.000000,14599.000000,14581.000000,14597.000000,14609.000000,14614.000000,14588.000000,14610.000000
mean,2.250240,79.300762,0.490739,33.455943,31.559488,1184.346084,0.023366,0.044664,4.812620,0.064083,2.122422,2.356629,3.155673,4.090143,0.234155
std,3.288042,179.374699,1.252376,140.146256,44.897089,2009.496307,0.050011,0.049912,16.887366,0.202583,0.914404,1.721823,2.405155,4.040147,0.423484
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,173.187500,0.000000,0.014502,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,0.000000
50%,1.000000,5.000000,0.000000,0.000000,17.500000,584.333333,0.003478,0.026406,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000,0.000000
75%,3.000000,91.000000,0.000000,0.000000,38.000000,1434.255128,0.018182,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000,1.000000


In [8]:
categorical_cols=list(my_data.select_dtypes(include="object").columns)
numerical_cols=list(my_data.select_dtypes(include="float").columns)
categorical_cols, numerical_cols

(['Month_SeasonalPurchase',
  'CustomerType',
  'Gender',
  'Cookies Setting',
  'Education',
  'Marital Status'],
 ['HomePage',
  'HomePage_Duration',
  'LandingPage',
  'LandingPage_Duration',
  'ProductDescriptionPage',
  'ProductDescriptionPage_Duration',
  'GoogleMetric:Bounce Rates',
  'GoogleMetric:Exit Rates',
  'GoogleMetric:Page Values',
  'SeasonalPurchase',
  'OS',
  'SearchEngine',
  'Zone',
  'Type of Traffic',
  'WeekendPurchase'])

In [9]:
#getting unique values and count of unique values in data
for col in categorical_cols:
  print("Unique values in",col,"are",my_data[col].unique(),"and its count is", len(my_data[col].unique().tolist()))

Unique values in Month_SeasonalPurchase are ['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec' nan] and its count is 11
Unique values in CustomerType are ['Returning_Visitor' 'New_Visitor' 'Other' nan] and its count is 4
Unique values in Gender are ['Not Specified' 'Female' 'Male' nan] and its count is 4
Unique values in Cookies Setting are ['Deny' 'ALL' 'Required' nan] and its count is 4
Unique values in Education are ['Not Specified' 'Others' 'Diploma' 'Graduate' nan] and its count is 5
Unique values in Marital Status are ['Other' 'Married' 'Single' nan] and its count is 4


In [10]:
null_counts=my_data.isnull().sum()
null_counts

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
Made_Purchase                        0
dtype: int64

In [11]:
#checking how much percentage is null values, if it is smaller, we can do imputing first, then encoding
print((153/my_data.shape[0])*100)

1.0386260267463172


In [12]:
#getting all ? as nan values to easily impute
my_data.replace("?", np.nan, inplace=True)
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   GoogleMetric:Bounce Rates        14580 non-null  float64
 7   GoogleMetric:Exit Rates          14602 non-null  float64
 8   GoogleMetric:Page Values         14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

In [13]:
null_counts=my_data.isnull().sum()
null_counts

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
Made_Purchase                        0
dtype: int64

In [14]:
#applying simple imputer on numerical and categorical variables separately with strategies median and mode respectively
from sklearn.impute import SimpleImputer
numerical_imputer=SimpleImputer(missing_values=np.nan, strategy="median")

for numerical_col in numerical_cols:
  my_data[numerical_col]=numerical_imputer.fit_transform(my_data[[numerical_col]]).reshape(-1)




categorical_imputer=SimpleImputer(missing_values=np.nan, strategy="most_frequent")

for categorical_col in categorical_cols:
  my_data[categorical_col]=categorical_imputer.fit_transform(my_data[[categorical_col]]).reshape(-1)


#The reshape(-1) method is used to flatten the array into a 1D array, while preserving the number of elements after imputation

In [15]:
my_data.isnull().sum()

HomePage                           0
HomePage_Duration                  0
LandingPage                        0
LandingPage_Duration               0
ProductDescriptionPage             0
ProductDescriptionPage_Duration    0
GoogleMetric:Bounce Rates          0
GoogleMetric:Exit Rates            0
GoogleMetric:Page Values           0
SeasonalPurchase                   0
Month_SeasonalPurchase             0
OS                                 0
SearchEngine                       0
Zone                               0
Type of Traffic                    0
CustomerType                       0
Gender                             0
Cookies Setting                    0
Education                          0
Marital Status                     0
WeekendPurchase                    0
Made_Purchase                      0
dtype: int64

In [16]:
for col in categorical_cols:
  print("Unique values in",col,"are",my_data[col].unique(),"and its count is", len(my_data[col].unique().tolist()))

Unique values in Month_SeasonalPurchase are ['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec'] and its count is 10
Unique values in CustomerType are ['Returning_Visitor' 'New_Visitor' 'Other'] and its count is 3
Unique values in Gender are ['Not Specified' 'Female' 'Male'] and its count is 3
Unique values in Cookies Setting are ['Deny' 'ALL' 'Required'] and its count is 3
Unique values in Education are ['Not Specified' 'Others' 'Diploma' 'Graduate'] and its count is 4
Unique values in Marital Status are ['Other' 'Married' 'Single'] and its count is 3


In [17]:
#encoding the categorical variables
#some with one hot encoding, some with ordinal encoding
#looking at data, it seems that we need to use one hot encoding only, because no variable has order in them

ohe=OneHotEncoder()

encoded_cols=ohe.fit_transform(my_data[categorical_cols].values)
encoded_cols_df = pd.DataFrame(encoded_cols.toarray(), columns=ohe.get_feature_names_out(categorical_cols))

# Combine the encoded columns with the original dataframe
my_data_encoded = pd.concat([my_data.drop(categorical_cols, axis=1), encoded_cols_df], axis=1)
my_data_encoded

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,Cookies Setting_ALL,Cookies Setting_Deny,Cookies Setting_Required,Education_Diploma,Education_Graduate,Education_Not Specified,Education_Others,Marital Status_Married,Marital Status_Other,Marital Status_Single
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.000000,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.4,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,1.0,4.000000,0.0,0.0,39.0,983.138889,0.015385,0.017599,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
14727,8.0,117.023809,2.0,57.0,11.0,252.892857,0.000000,0.011078,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
14728,2.0,75.600000,2.0,652.8,10.0,1143.666667,0.000000,0.023333,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
14729,0.0,0.000000,0.0,0.0,6.0,1057.000000,0.000000,0.033333,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [18]:
my_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 42 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14731 non-null  float64
 1   HomePage_Duration                14731 non-null  float64
 2   LandingPage                      14731 non-null  float64
 3   LandingPage_Duration             14731 non-null  float64
 4   ProductDescriptionPage           14731 non-null  float64
 5   ProductDescriptionPage_Duration  14731 non-null  float64
 6   GoogleMetric:Bounce Rates        14731 non-null  float64
 7   GoogleMetric:Exit Rates          14731 non-null  float64
 8   GoogleMetric:Page Values         14731 non-null  float64
 9   SeasonalPurchase                 14731 non-null  float64
 10  OS                               14731 non-null  float64
 11  SearchEngine                     14731 non-null  float64
 12  Zone              

In [19]:


label_encoder=LabelEncoder()
my_data_encoded["Made_Purchase"]=label_encoder.fit_transform(my_data_encoded["Made_Purchase"])


In [20]:
my_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 42 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14731 non-null  float64
 1   HomePage_Duration                14731 non-null  float64
 2   LandingPage                      14731 non-null  float64
 3   LandingPage_Duration             14731 non-null  float64
 4   ProductDescriptionPage           14731 non-null  float64
 5   ProductDescriptionPage_Duration  14731 non-null  float64
 6   GoogleMetric:Bounce Rates        14731 non-null  float64
 7   GoogleMetric:Exit Rates          14731 non-null  float64
 8   GoogleMetric:Page Values         14731 non-null  float64
 9   SeasonalPurchase                 14731 non-null  float64
 10  OS                               14731 non-null  float64
 11  SearchEngine                     14731 non-null  float64
 12  Zone              

In [21]:
y = my_data_encoded['Made_Purchase']

In [22]:

my_data_new = my_data_encoded.drop('Made_Purchase', axis=1)

# selector = SelectKBest(chi2, k=22)
# my_data_selected = selector.fit_transform(my_data_new, y)

In [23]:
# feature_names=my_data.columns
# feature_names.tolist()

In [24]:
# my_data_select=pd.DataFrame(my_data_selected, columns=selector.get_feature_names_out(selector.feature_names_in_))
# my_data_select

In [25]:
# selected_cols=selector.get_feature_names_out(selector.feature_names_in_)
# selected_cols

In [26]:
scaler = StandardScaler()
for numerical_col in numerical_cols:
    my_data_new[numerical_col]=scaler.fit_transform(my_data_new[numerical_col].values.reshape(-1,1))

Random forest

In [27]:
# X_train, X_test, y_train, y_test=train_test_split(my_data_new, y, test_size=0.2, random_state=42, shuffle=True)

# rfc=RandomForestClassifier(max_depth=6, max_leaf_nodes=6, n_estimators=100, random_state=0, bootstrap=False, criterion="gini" )
# rfc.fit(X_train, y_train)
# y_pred=rfc.predict(X_test)

# f1=f1_score(y_test, y_pred)
# accu=accuracy_score(y_test, y_pred)

# print("f1 score:",f1)
# print("accuracy:",accu)
# print(classification_report(y_test, y_pred))

In [28]:



# X_train, X_test, y_train, y_test=train_test_split(my_data_new, y, test_size=0.2, random_state=42, shuffle=True)


# rf=RandomForestClassifier()
# rf.fit(X_train, y_train)
# y_pred=rf.predict(X_test)

# f1=f1_score(y_test, y_pred)
# accu=accuracy_score(y_test, y_pred)

# print(classification_report(y_test, y_pred))


# print("f1 score:",f1)
# print("accuracy:",accu)

Logistic Regression

In [29]:


# X_train, X_test, y_train, y_test=train_test_split(my_data_new, y, test_size=0.2, random_state=42, shuffle=True)


# lr=LogisticRegression(max_iter=5000, solver="saga", penalty="l1")
# lr.fit(X_train, y_train)
# y_pred=lr.predict(X_test)

# f1=f1_score(y_test, y_pred)
# accu=accuracy_score(y_test, y_pred)


# print("f1 score:",f1)
# print("accuracy:",accu)

> Decision Tree Classifier

In [30]:
X_train, X_test, y_train, y_test=train_test_split(my_data_new, y, test_size=0.2, random_state=42, shuffle=True)

dt=DecisionTreeClassifier(max_depth=3, min_samples_split=5, min_samples_leaf=10, max_features=30, random_state=0)

dt.fit(X_train, y_train)
y_pred=dt.predict(X_test)
scre=dt.score(X_train, y_train)
f1=f1_score(y_test, y_pred, average="micro")

print("f1 score:",f1)
print("Training Set Score",scre)

f1 score: 0.6851034950797421
Training Set Score 0.6623387644263408


# Testing Set

In [31]:
testing=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
testing

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,OS,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,1.0,1.0,1.0,Returning_Visitor,Not Specified,Deny,Not Specified,Single,0.0
1,0.0,0.000000,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,...,2.0,2.0,1.0,2.0,Returning_Visitor,Not Specified,ALL,Graduate,Married,0.0
2,0.0,0.000000,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,...,2.0,2.0,1.0,3.0,Returning_Visitor,Female,Required,Diploma,Other,0.0
3,0.0,0.000000,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,...,2.0,2.0,2.0,3.0,Returning_Visitor,Not Specified,Required,Graduate,Other,0.0
4,0.0,0.000000,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,...,1.0,1.0,4.0,3.0,Returning_Visitor,Female,Deny,Others,Other,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.0,7.0,208.000000,0.000000,0.028571,0.0,0.0,...,4.0,1.0,1.0,5.0,Returning_Visitor,Not Specified,Required,Graduate,Other,1.0
6595,0.0,0.000000,3.0,44.0,179.0,1738.472529,0.000027,0.025998,0.0,0.0,...,2.0,4.0,9.0,11.0,Returning_Visitor,Male,Required,Not Specified,Other,0.0
6596,5.0,99.166667,1.0,27.0,33.0,NaN,0.002778,0.009127,0.0,0.6,...,8.0,5.0,1.0,2.0,Returning_Visitor,Female,Required,Graduate,Married,0.0
6597,0.0,0.000000,0.0,0.0,3.0,9.000000,0.066667,0.133333,0.0,0.0,...,2.0,2.0,2.0,3.0,Returning_Visitor,Not Specified,Deny,Diploma,Other,1.0


In [32]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         6548 non-null   float64
 1   HomePage_Duration                6544 non-null   float64
 2   LandingPage                      6543 non-null   float64
 3   LandingPage_Duration             6524 non-null   float64
 4   ProductDescriptionPage           6541 non-null   float64
 5   ProductDescriptionPage_Duration  6536 non-null   float64
 6   GoogleMetric:Bounce Rates        6533 non-null   float64
 7   GoogleMetric:Exit Rates          6548 non-null   float64
 8   GoogleMetric:Page Values         6554 non-null   float64
 9   SeasonalPurchase                 6554 non-null   float64
 10  Month_SeasonalPurchase           6541 non-null   object 
 11  OS                               6537 non-null   float64
 12  SearchEngine        

In [33]:
#getting unique values and count of unique values in data
print("Unique values in Month_SeasonalPurchase are",testing["Month_SeasonalPurchase"].unique(),"and its count is", len(testing["Month_SeasonalPurchase"].unique().tolist()))

Unique values in Month_SeasonalPurchase are ['Feb' 'Mar' 'May' 'June' 'Oct' 'Nov' 'Sep' 'Aug' 'Jul' 'Dec' nan] and its count is 11


In [34]:
categorical_cols_testing=list(testing.select_dtypes(include="object").columns)
numerical_cols_testing=list(testing.select_dtypes(include="float").columns)
categorical_cols_testing, numerical_cols_testing

(['Month_SeasonalPurchase',
  'CustomerType',
  'Gender',
  'Cookies Setting',
  'Education',
  'Marital Status'],
 ['HomePage',
  'HomePage_Duration',
  'LandingPage',
  'LandingPage_Duration',
  'ProductDescriptionPage',
  'ProductDescriptionPage_Duration',
  'GoogleMetric:Bounce Rates',
  'GoogleMetric:Exit Rates',
  'GoogleMetric:Page Values',
  'SeasonalPurchase',
  'OS',
  'SearchEngine',
  'Zone',
  'Type of Traffic',
  'WeekendPurchase'])

In [35]:
for col in categorical_cols_testing:
  print("Unique values in",col,"are",testing[col].unique(),"and its count is", len(testing[col].unique().tolist()))

Unique values in Month_SeasonalPurchase are ['Feb' 'Mar' 'May' 'June' 'Oct' 'Nov' 'Sep' 'Aug' 'Jul' 'Dec' nan] and its count is 11
Unique values in CustomerType are ['Returning_Visitor' 'New_Visitor' 'Other' nan] and its count is 4
Unique values in Gender are ['Not Specified' 'Female' 'Male' nan] and its count is 4
Unique values in Cookies Setting are ['Deny' 'ALL' 'Required' nan] and its count is 4
Unique values in Education are ['Not Specified' 'Graduate' 'Diploma' 'Others' nan] and its count is 5
Unique values in Marital Status are ['Single' 'Married' 'Other' nan] and its count is 4


In [36]:
null_counts_testing=testing.isnull().sum()
null_counts_testing

HomePage                           51
HomePage_Duration                  55
LandingPage                        56
LandingPage_Duration               75
ProductDescriptionPage             58
ProductDescriptionPage_Duration    63
GoogleMetric:Bounce Rates          66
GoogleMetric:Exit Rates            51
GoogleMetric:Page Values           45
SeasonalPurchase                   45
Month_SeasonalPurchase             58
OS                                 62
SearchEngine                       58
Zone                               47
Type of Traffic                    68
CustomerType                       88
Gender                             64
Cookies Setting                    62
Education                          69
Marital Status                     47
WeekendPurchase                    58
dtype: int64

In [37]:
#checking how much percentage is null values, if it is smaller, we can do imputing first, then encoding
print((88/testing.shape[0])*100)

1.3335353841491135


In [38]:
#getting all ? as nan values to easily impute
testing.replace("?", np.nan, inplace=True)
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         6548 non-null   float64
 1   HomePage_Duration                6544 non-null   float64
 2   LandingPage                      6543 non-null   float64
 3   LandingPage_Duration             6524 non-null   float64
 4   ProductDescriptionPage           6541 non-null   float64
 5   ProductDescriptionPage_Duration  6536 non-null   float64
 6   GoogleMetric:Bounce Rates        6533 non-null   float64
 7   GoogleMetric:Exit Rates          6548 non-null   float64
 8   GoogleMetric:Page Values         6554 non-null   float64
 9   SeasonalPurchase                 6554 non-null   float64
 10  Month_SeasonalPurchase           6541 non-null   object 
 11  OS                               6537 non-null   float64
 12  SearchEngine        

In [39]:
#applying simple imputer on numerical and categorical variables separately with strategies median and mode respectively
from sklearn.impute import SimpleImputer
numerical_imputer=SimpleImputer(missing_values=np.nan, strategy="median")

for numerical_col in numerical_cols_testing:
  testing[numerical_col]=numerical_imputer.fit_transform(testing[[numerical_col]]).reshape(-1)




categorical_imputer=SimpleImputer(missing_values=np.nan, strategy="most_frequent")

for categorical_col in categorical_cols_testing:
  testing[categorical_col]=categorical_imputer.fit_transform(testing[[categorical_col]]).reshape(-1)


#The reshape(-1) method is used to flatten the array into a 1D array, while preserving the number of elements after imputation

In [40]:
testing.isnull().sum()

HomePage                           0
HomePage_Duration                  0
LandingPage                        0
LandingPage_Duration               0
ProductDescriptionPage             0
ProductDescriptionPage_Duration    0
GoogleMetric:Bounce Rates          0
GoogleMetric:Exit Rates            0
GoogleMetric:Page Values           0
SeasonalPurchase                   0
Month_SeasonalPurchase             0
OS                                 0
SearchEngine                       0
Zone                               0
Type of Traffic                    0
CustomerType                       0
Gender                             0
Cookies Setting                    0
Education                          0
Marital Status                     0
WeekendPurchase                    0
dtype: int64

In [41]:
#encoding the categorical variables
#some with one hot encoding, some with ordinal encoding
#looking at data, it seems that we need to use one hot encoding only, because no variablehas order in them

from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
# my_data[categorical_cols].values.reshape(-1,1)
encoded_cols=ohe.fit_transform(testing[categorical_cols_testing].values)
encoded_cols_df = pd.DataFrame(encoded_cols.toarray(), columns=ohe.get_feature_names_out(categorical_cols_testing))

# Combine the encoded columns with the original dataframe
testing_encoded = pd.concat([testing.drop(categorical_cols, axis=1), encoded_cols_df], axis=1)
testing_encoded

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,Cookies Setting_ALL,Cookies Setting_Deny,Cookies Setting_Required,Education_Diploma,Education_Graduate,Education_Not Specified,Education_Others,Marital Status_Married,Marital Status_Other,Marital Status_Single
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.000000,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.000000,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.000000,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.0,7.0,208.000000,0.000000,0.028571,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6595,0.0,0.000000,3.0,44.0,179.0,1738.472529,0.000027,0.025998,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6596,5.0,99.166667,1.0,27.0,33.0,570.341614,0.002778,0.009127,0.0,0.6,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6597,0.0,0.000000,0.0,0.0,3.0,9.000000,0.066667,0.133333,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


# selector = SelectKBest(chi2, k=16)
# my_data_selected = selector.fit_transform(my_data_new, y)

# Scaling the features
scaler = StandardScaler()
for numerical_col in numerical_cols_testing:
    testing_encoded[numerical_col]=scaler.fit_transform(testing_encoded[numerical_col].values.reshape(-1,1))


In [43]:
y_testing_pred=dt.predict(testing_encoded)


In [44]:
final=pd.DataFrame()

In [45]:
final["id"]=np.arange(0,6599)
final["Made_Purchase"]=y_testing_pred

In [46]:
final.head()

,id,Made_Purchase
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [47]:
final["Made_Purchase"]=final["Made_Purchase"].astype("bool")
final

,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False


In [48]:
final.to_csv("submission.csv", index=False)